In [344]:
#Imports and Settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from datetime import datetime
from operator import mul
import seaborn as sn
import random

#Pandas Settings to Display Rows and Cols
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 10) 

#Matplotlib Style Settings
plt.style.use('fivethirtyeight')
params = {'legend.fontsize': 'large',
          'figure.figsize': (15, 10),
         'axes.labelsize': 'large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
plt.rcParams.update(params)

#Pandas Standaridzer for Float Formatting
pd.options.display.float_format = '{:,.2f}'.format

import warnings
warnings.filterwarnings("ignore")

from tqdm.auto import tqdm
tqdm.pandas(desc="my bar!")

In [67]:
# Looping through data folder to concat CSV's into one big dataframe
# 986720 rows in dataset
# 63 columns
# Reinitialize HERE

for i,name in enumerate(glob.glob('data/CSV/*')):
    if i == 0:
        df = pd.read_csv(name)
    df2 = pd.read_csv(name)
    concat = pd.concat([df,df2],ignore_index=True)
    df = concat

In [68]:
#Back Up Reinitilization 
data = df.copy()

In [353]:
# Reinitilize DF
df = data.copy()

In [369]:
samp = pd.read_csv('data/Sample DP - Sheet1.csv')
geo = samp[['Theatre','Division','Branch','TV Market','Area']]
len(geo['Theatre'].unique())

2754

In [367]:
len(df['Theatre'].unique())

6026

In [249]:
#Trimming off extraneous columns
df = df.iloc[:,1:31]
df = df.drop(['LA #','Classics Request ID','End Of Run','Play Dates','Start Date','Unnamed: 24','Local','Boxoffice Net','Film Rental','To Receive','FRM%','MG/FF','Price','Customer Reference','Office'],axis=1)
df.head(2)

,T#,First Date,Last Date,FRM,Release,Media Formats,Rental Measure,Theatre,Status,Boxoffice,Admissions,Terms%,Film Buyer,Buying Circuit,Payer
0,16.00,5/4/2018,5/10/2018,sp,12 STRONG,ENG.2.DIG.0.0.0.0,Default,LUCAS CINEMA 10,Returns In,562.74,86,44.83,"Sailors, Shannon",AMERICAN MULTI-CINEMA,AMERICAN MULTI-CINEMA
1,15.00,4/27/2018,5/3/2018,sp,12 STRONG,ENG.2.DIG.0.0.0.0,Default,LUCAS CINEMA 10,Returns In,302.28,47,44.83,"Sailors, Shannon",AMERICAN MULTI-CINEMA,AMERICAN MULTI-CINEMA


In [250]:
#Dropping any non SP payments
df = df[df['FRM']=='sp'] 

In [251]:
# Only looking at records with grosses returned
df = df[df['Status']=='Returns In'];

In [252]:
#Dropping 500 NAN media formats
df = df.dropna(axis=0)

In [262]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985328 entries, 0 to 986718
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   T#              985328 non-null  float64       
 1   First Date      985328 non-null  datetime64[ns]
 2   Last Date       985328 non-null  datetime64[ns]
 3   FRM             985328 non-null  object        
 4   Release         985328 non-null  object        
 5   Media Formats   985328 non-null  object        
 6   Rental Measure  985328 non-null  object        
 7   Theatre         985328 non-null  object        
 8   Status          985328 non-null  object        
 9   Boxoffice       985328 non-null  float64       
 10  Admissions      985328 non-null  float64       
 11  Terms%          985328 non-null  object        
 12  Film Buyer      985328 non-null  object        
 13  Buying Circuit  985328 non-null  object        
 14  Payer           985328 non-null  obj

In [254]:
df['First Date'] = pd.to_datetime(df['First Date'])
df['Last Date'] = pd.to_datetime(df['Last Date'])

In [255]:
df['Boxoffice'] = df['Boxoffice'].apply(lambda x: x.replace(',', ''))
df['Boxoffice'] = df['Boxoffice'].astype(float)

In [256]:
df['Admissions'] = df['Admissions'].apply(lambda x: x.replace(',', ''))
df['Admissions'] = df['Admissions'].astype(float)


In [257]:
df['Media Formats'] = df['Media Formats'].astype(str)

In [258]:
def format_assign(x):
    if 'IMAX' in x or 'IXD'in x or 'DIMX' in x:     
        return 'IMAX'
    if 'PLF' in x or 'PLD' in x:
        return 'PLF'
    elif '3D' in x:
        return '3D'
    else:
        return '2D'

In [259]:
df['Media Formats'] = df['Media Formats'].apply(format_assign)

In [260]:
df['Media Formats'].unique()

array(['2D', 'PLF', 'IMAX', '3D'], dtype=object)

In [261]:
df.groupby('Media Formats').sum('Boxoffice')

,T#,Boxoffice,Admissions
Media Formats,,,
2D,"3,152,523.00","3,838,437,835.45","443,754,993.00"
3D,"23,626.00","202,924,047.01","19,580,021.00"
IMAX,"86,211.00","600,785,403.66","48,318,338.00"
PLF,"117,065.00","806,759,219.27","73,701,768.00"


In [226]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df.isna().sum())

T#                0
First Date        0
Last Date         0
FRM               0
Release           0
Media Formats     0
Rental Measure    0
Theatre           0
Status            0
Boxoffice         0
Admissions        0
Terms%            0
Film Buyer        0
Buying Circuit    0
Payer             0
dtype: int64


In [263]:
df.head(1)

,T#,First Date,Last Date,FRM,Release,Media Formats,Rental Measure,Theatre,Status,Boxoffice,Admissions,Terms%,Film Buyer,Buying Circuit,Payer
0,16.00,2018-05-04,2018-05-10,sp,12 STRONG,2D,Default,LUCAS CINEMA 10,Returns In,562.74,86.00,44.83,"Sailors, Shannon",AMERICAN MULTI-CINEMA,AMERICAN MULTI-CINEMA


In [88]:
test = df.query("Theatre=='LUCAS CINEMA 10' & Release=='12 STRONG'").sort_values('T#')
test

,T#,Start Date,First Date,Last Date,FRM,Release,Media Formats,Rental Measure,Theatre,Status,Boxoffice,Admissions,Terms%,Film Buyer,Buying Circuit,Payer
10646,1.00,1/18/2018,1/18/2018,1/25/2018,sp,12 STRONG,ENG.2.DIG.0.0.0.0,Default,LUCAS CINEMA 10,Returns In,"6,684.22",970,44.83,"PRIVITERA, KAITLIN",AMERICAN MULTI-CINEMA,AMERICAN MULTI-CINEMA
23288,1.00,1/18/2018,1/18/2018,1/25/2018,sp,12 STRONG,ENG.2.DIG.0.0.0.0,Default,LUCAS CINEMA 10,Returns In,"6,684.22",970,44.83,"PRIVITERA, KAITLIN",AMERICAN MULTI-CINEMA,AMERICAN MULTI-CINEMA
20505,2.00,1/26/2018,1/26/2018,2/1/2018,sp,12 STRONG,ENG.2.DIG.0.0.0.0,Default,LUCAS CINEMA 10,Returns In,"3,843.22",560,44.83,"PRIVITERA, KAITLIN",AMERICAN MULTI-CINEMA,AMERICAN MULTI-CINEMA
7863,2.00,1/26/2018,1/26/2018,2/1/2018,sp,12 STRONG,ENG.2.DIG.0.0.0.0,Default,LUCAS CINEMA 10,Returns In,"3,843.22",560,44.83,"PRIVITERA, KAITLIN",AMERICAN MULTI-CINEMA,AMERICAN MULTI-CINEMA
4916,3.00,2/2/2018,2/2/2018,2/8/2018,sp,12 STRONG,ENG.2.DIG.0.0.0.0,Default,LUCAS CINEMA 10,Returns In,"1,906.47",284,44.83,"PRIVITERA, KAITLIN",AMERICAN MULTI-CINEMA,AMERICAN MULTI-CINEMA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12646,14.00,4/20/2018,4/20/2018,4/26/2018,sp,12 STRONG,ENG.2.DIG.0.0.0.0,Default,LUCAS CINEMA 10,Returns In,458.01,71,44.83,"Sailors, Shannon",AMERICAN MULTI-CINEMA,AMERICAN MULTI-CINEMA
1,15.00,4/27/2018,4/27/2018,5/3/2018,sp,12 STRONG,ENG.2.DIG.0.0.0.0,Default,LUCAS CINEMA 10,Returns In,302.28,47,44.83,"Sailors, Shannon",AMERICAN MULTI-CINEMA,AMERICAN MULTI-CINEMA
12643,15.00,4/27/2018,4/27/2018,5/3/2018,sp,12 STRONG,ENG.2.DIG.0.0.0.0,Default,LUCAS CINEMA 10,Returns In,302.28,47,44.83,"Sailors, Shannon",AMERICAN MULTI-CINEMA,AMERICAN MULTI-CINEMA
12642,16.00,5/4/2018,5/4/2018,5/10/2018,sp,12 STRONG,ENG.2.DIG.0.0.0.0,Default,LUCAS CINEMA 10,Returns In,562.74,86,44.83,"Sailors, Shannon",AMERICAN MULTI-CINEMA,AMERICAN MULTI-CINEMA


# Anonymizing Data

In [281]:
from faker import Faker

In [266]:
# Number of values to annoymize
# 6022 Theatres
# 4661 Film Buyer/Buying Circuit/Payer
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df[['Theatre','Film Buyer','Buying Circuit','Payer']].nunique())

Theatre           6022
Film Buyer        1052
Buying Circuit     802
Payer             2807
dtype: int64


In [309]:
Faker.seed(2)
fake = Faker()
fake_names = []
for _ in tqdm(range(5300)):
    fake_names.append(fake.company())

In [310]:
len(set(fake_names))

4795

In [324]:
cinema_names = pd.read_csv('data/Cinema_names.csv',header=None)

In [323]:
cinema_names = list(cinema_names[0].unique())

In [342]:
list = []
for n in tqdm(range(0,674)):
    if len(cinema_names[0].apply(lambda x: x.split(' '))[n])>1:
        list.append(cinema_names[0].apply(lambda x: x.split(' '))[n][1])
    else:
        continue

In [345]:
suffixes = ['Theaters','Cinemas','Movies','Flicks','Screens','Forum']

In [352]:
random_theatre_names = 

random.choice(list) +' '+ random.choice(list) + ' '+ random.choice(suffixes) 

'Foster Option Flicks'

In [375]:
maccs_theatres = df['Theatre'].unique().tolist()

In [383]:
maccs_theatres_dict = dict.fromkeys(maccs_theatres)

In [388]:
for key,value in maccs_theatres_dict.items():
    maccs_theatres_dict[key] = random.choice(list) +' '+ random.choice(list) + ' '+ random.choice(suffixes) 

In [389]:
maccs_theatres_dict

{'LUCAS CINEMA 10': 'Saga Dare Flicks',
 'MOVIES 12': 'Magnolia Hacks Cinemas',
 'FIESTA 16 STADIUM': 'Serendipity Bloom Movies',
 'OMNI CINEMA 8': 'Wave Blink Cinemas',
 'RAINBOW REGINA 9': 'Create Bold Screens',
 'MARCUS SHOWTIME 12': 'Awry Sprightly Flicks',
 'MONTWOOD 7': 'Dreaming Prime Screens',
 'PARKWAY 8': 'Flower Incorporated Movies',
 'CENTURY GATEWAY 12': 'Arid Radiant Movies',
 'MOVIES 10': 'Radical Proxy Flicks',
 'KENDIG MOVIES 6': 'Incorporated Huge Forum',
 'HOLLYWOOD USA 15': 'Kingdom Buyer Cinemas',
 'NASA 8': 'Trident Greatest Movies',
 'PREMIERE 6': 'Maps Myna Forum',
 'THE PICTURE SHOW 6': 'Buyer Ahead Cinemas',
 'PICTURE SHOW 8 @ SUPERSTITION SPRINGS': 'Bard Explosion Flicks',
 'CLOVIS MOVIES 8': 'Crimson Question Flicks',
 'MOVIE MILL 7': 'Advisor Delight Cinemas',
 'DOLLAR 3': 'Bevy Edge Forum',
 'SHOWPLACE SOUTH 6': 'Collective Number Movies',
 'VENUE 8 CINEMAS': 'Sanguine Basement Movies',
 'UNIVERSITY 3 MALL': 'Link Collective Cinemas',
 'PICTURE SHOW THEATR